参考：https://github.com/XDTD/fund_crawler
http://fund.eastmoney.com/pingzhongdata/000001.js
公司列表(Github上直接打开好像会提示not found,复制到浏览器上方直接进入即可)：http://fund.eastmoney.com/js/jjjz_gs.js
基金列表：http://fund.eastmoney.com/js/fundcode_search.js
基金信息1：http://fund.eastmoney.com/pingzhongdata/'+code+'.js‘ 其中,code为6位整数，如000001的URL位=为http://fund.eastmoney.com/pingzhongdata/000001.js
基金信息2:http://fund.eastmoney.com/f10/tsdata_'+code+'.html'，同上
基金经理信息:http://fundf10.eastmoney.com/jjjl_'+code+'.html',同上


In [11]:
import requests
import re
import yaml
import logging
import json
logging.basicConfig(format='%(levelname)s : %(message)s',
                    level=logging.DEBUG,
                    handlers=[logging.StreamHandler()])


logger = logging.getLogger(__name__)


def get_resonse(url):
    """
    :param url: 网页URL
    :return: 爬取的文本信息
    """
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        print('Failed to get response to url!')
        return ''

def get_fund_info(code):
    failed_list = []
    data_list = {}
    url = 'http://fund.eastmoney.com/pingzhongdata/'+code+'.js'
    response = get_resonse(url)
    # 爬取失败等待再次爬取
    if response is '':
        return ''
    else:
        strs = re.findall(r'var(.*?);',response)
        for i in range(0,len(strs)):
            tmp = strs[i].split('=')
            var_name = tmp[0].strip()
            data_list[var_name] = [tmp[1]]
        return data_list

data = get_fund_info("151001")

print(type(data['Data_currentFundManager'][0]))

data = data['Data_currentFundManager'][0]
data = json.loads(data)[0]

print(data['id'])
print(data['workTime'])
print(data['fundSize'])

id=data['id']

DEBUG : Starting new HTTP connection (1): fund.eastmoney.com
DEBUG : http://fund.eastmoney.com:80 "GET /pingzhongdata/151001.js HTTP/1.1" 200 106151


<class 'str'>
30413768
5年又242天
23.54亿(3只基金)
